In [40]:
from sklearn.model_selection import KFold
from sklearn import metrics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
from sklearn import ensemble
from scipy import stats

def mse(predictions, targets):
    count = 0
    k = []
    for x in predictions:
        temp = (((predictions[count] - targets[count]) ** 2))
        count+=1
        k.append(temp)
    return k

# df = pd.read_csv('drop_out.csv')
df = pd.read_csv('fill.csv')

# X = df[['Weight','PCV','PCV\ndonor','Volume']]
# print(X.columns)
Vet = df['PCV_target']

X = df[['Weight', 'PCV', 'PCV\ndonor', 'Volume', 'WBC', 'PLT\n______', 'HGB', 'RBC', 'MCV', 'MCHC', 'MCH',
        'SEGS', 'LYMPH', 'MONO','RDW']]
# X = df[['Weight', 'PCV', 'PCV\ndonor', 'Volume', 'WBC','PLT\n______','HGB' , 'RBC','MCV']]
y = df['PCV_afterdonation']
column_name = X.columns

# # convert to numeric
# X = X.apply(pd.to_numeric, errors='coerce')
# y = y.apply(pd.to_numeric, errors='coerce')
# Vet = Vet.apply(pd.to_numeric, errors='coerce')


# # fill with mean
# X = X.fillna(X.mean())
# y = y.fillna(y.mean())
# Vet = Vet.fillna(Vet.mean())


X = np.array(X)
# y = df['attack']
y = np.array(y)
Vet = np.array(Vet)


scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


kf = KFold(n_splits=10)
kf.get_n_splits(X)

MSE=[]

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#     lm = LinearRegression()
#     lm.fit(X_train,y_train)
    model = ensemble.GradientBoostingRegressor(random_state=1)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    temp_MSE = (metrics.mean_squared_error(y_test, predictions))
    MSE.append(temp_MSE)
#     print(temp_MSE)


    
print ('Average MSE 10 fold cross validation Xg: %f'%np.mean(MSE))


MSE=[]
train_MSE=[]
# print("leave one out")

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
loo.get_n_splits(X)

# print(loo)
coef_list = []
for train_index, test_index in loo.split(X):
#    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
#    print(X_train, X_test, y_train, y_test)

    model = ensemble.GradientBoostingRegressor(random_state=1,max_depth=1,n_estimators=300)
    model.fit(X_train, y_train)
    
        #     for train accuracy
    xxx = model.predict(X_train)
    temp_train_MSE = (metrics.mean_squared_error(y_train, xxx))
    train_MSE.append(temp_train_MSE)
#     asdfafdafd

    predictions = model.predict(X_test)
    temp_MSE = (metrics.mean_squared_error(y_test, predictions))
    MSE.append(temp_MSE)
#     print(lm.coef_)
#     coef_list.append(lm.coef_)
#     print(temp_MSE)
print ('Average Xg Leave one out :%f ± %f'%(np.mean(MSE), np.std(MSE)))
print ('average train MSE  : %f ± %f'%(np.mean(train_MSE), np.std(train_MSE)))
print("********")
mse_vet = mse(Vet,y)
# print(mse_vet)
# print(stats.ttest_rel(MSE,mse_vet).pvalue)
p_value = stats.ttest_rel(MSE,mse_vet).pvalue
print('P value is %f'%p_value)
print ('MSE Vet : %f ± %f'%(np.mean(metrics.mean_squared_error(Vet, y)), np.std(mse_vet)))
print (column_name)
print(mse_vet)
print(metrics.mean_squared_error(28.6274, 23.6))

Average MSE 10 fold cross validation Xg: 46.726681
Average Xg Leave one out :36.489647 ± 63.705802
average train MSE  : 4.092954 ± 0.123901
********
P value is 0.650274
MSE Vet : 41.892226 ± 79.842624
Index(['Weight', 'PCV', 'PCV\ndonor', 'Volume', 'WBC', 'PLT\n______', 'HGB',
       'RBC', 'MCV', 'MCHC', 'MCH', 'SEGS', 'LYMPH', 'MONO', 'RDW'],
      dtype='object')
[25.275263356302936, 45.96144642378438, 141.02069994565917, 19.839600724138883, 9.6099999999999639, 3.9373744205537906, 37.210000000000015, 19.041322282314027, 40.23028778764381, 0.011327930272562445, 9.5635098007064094, 2.9265957985307569, 12.484444420888874, 19.102225828070861, 0.00073442463407280578, 0.99260631739369176, 0.27557132661325456, 2.8786777890888855, 61.636272652462331, 127.69000000000001, 0.62234568076543062, 23.541348352238725, 4.6738321913106526, 2.6909254144483215, 3.6131279464926329e-05, 34.238011160188869, 0.036113487459539961, 0.00022956855831039583, 5.0331947030926267, 19.975933602246915, 153.331656774

TypeError: Expected sequence or array-like, got <class 'float'>